In [7]:
import re
from collections import defaultdict

def parse_formula(formula):
    match = re.match(r"\[(.+)\](\d+)\s*(.*)", formula)
    if not match:
        raise ValueError("Format de formule incorrect")
    inside_block, global_percent, outside_elements = match.groups()
    global_percent = int(global_percent)

    result = defaultdict(float)

    # Fonction pour traiter un mini bloc
    def process_block(block_text, block_percent):
        parts = re.findall(r"\(([A-Za-z0-9\s]+)\)(\d+)|([A-Z][a-z]*)(\d+)", block_text)
        for part in parts:
            if part[0]:  # C'est un sous-bloc ( ... )%
                inner_block, inner_percent = part[0], int(part[1])
                new_block_percent = block_percent * inner_percent / 100
                inner_elements = re.findall(r"([A-Z][a-z]*)(\d+)", inner_block)
                for elem, perc in inner_elements:
                    perc = int(perc)
                    result[elem] += new_block_percent * perc / 100
            else:  # C'est un élément simple
                elem, perc = part[2], int(part[3])
                result[elem] += block_percent * perc / 100

    # Traiter le bloc principal
    process_block(inside_block, global_percent)

    # Traiter les éléments en dehors du bloc
    outside = re.findall(r"([A-Z][a-z]*)(\d+)", outside_elements)
    for elem, perc in outside:
        result[elem] += int(perc)

    return dict(result)

def format_result(result):
    return " ".join(f"{elem}{round(percent, 4)}" for elem, percent in result.items())

# Exemple d'utilisation
#formula = "[(Fe90 Co10)75 B20 Si5]96 Nb4"
formula = "[(Fe70 Co30)72 B24 Mo4]94 Dy6"
parsed = parse_formula(formula)
output = format_result(parsed)

print(output)

Fe47.376 Co20.304 B22.56 Mo3.76 Dy6.0


In [8]:
import re
from collections import defaultdict

def parse_formula(formula):
    result = defaultdict(float)

    def process_block(block_text, block_percent):
        pattern = r"\(([^\(\)]+)\)(\d+)|([A-Z][a-z]*)(\d+)"
        parts = re.findall(pattern, block_text)
        for part in parts:
            if part[0]:  # Sous-bloc trouvé
                inner_block, inner_percent = part[0], int(part[1])
                new_percent = block_percent * inner_percent / 100
                process_block(inner_block, new_percent)
            else:  # Élément simple trouvé
                elem, perc = part[2], int(part[3])
                result[elem] += block_percent * perc / 100

    # Check si formule commence par [ ]
    if formula.startswith('['):
        match = re.match(r"\[(.+)\](\d+)\s*(.*)", formula)
        if not match:
            raise ValueError("Format de formule incorrect")
        inside_block, global_percent, outside_elements = match.groups()
        global_percent = int(global_percent)

        process_block(inside_block, global_percent)

        # Traiter les éléments en dehors du bloc
        outside = re.findall(r"([A-Z][a-z]*)(\d+)", outside_elements)
        for elem, perc in outside:
            result[elem] += int(perc)

    else:
        # Formule sans crochets []
        process_block(formula, 100)

    return dict(result)

def format_result(result):
    # Trier par ordre alphabétique
    return " ".join(f"{elem}{round(percent, 4)}" for elem, percent in sorted(result.items()))

# Exemple d'utilisation
formulas = [
    "[(Fe70 Co30)72 B24 Mo4]94 Dy6",
    "(Fe70 Tb5 Si5 B20)96 Nb4",
    "Fe76 Mo4(P45 C20 B20 Si15)20"
]

for formula in formulas:
    parsed = parse_formula(formula)
    output = format_result(parsed)
    print(f"{formula} → {output}")

[(Fe70 Co30)72 B24 Mo4]94 Dy6 → B22.56 Co20.304 Dy6.0 Fe47.376 Mo3.76
(Fe70 Tb5 Si5 B20)96 Nb4 → B19.2 Fe67.2 Nb4.0 Si4.8 Tb4.8
Fe76 Mo4(P45 C20 B20 Si15)20 → B4.0 C4.0 Fe76.0 Mo4.0 P9.0 Si3.0
